# Markov Chain Ensemble Using Recom

@author: taiyyoson

### Using 2018 Election Data

In [ ]:
import pandas as pd
import geopandas as gpd
import maup
import matplotlib.pyplot as plt
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.tree import bipartition_tree
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
from gerrychain.metrics import efficiency_gap

import time
import os

In [3]:
## making graph
sc_graph = Graph.from_file("./SC/SC.shp")
sc_gdf = gpd.read_file("./SC/SC.shp")
sc_from_gpd_graph = Graph.from_geodataframe(sc_gdf, adjacency="rook") ## creating graph from geodataframe

In [10]:
## printing graph node information fields
sc_graph.nodes[1]

{'boundary_node': False,
 'area': 8648536.552029321,
 'COUNTY': '013',
 'PCODE': '111',
 'CODE_NAME': 'Burton 1B',
 'G20PRER': 160,
 'G20PRED': 650,
 'G20USSR': 142,
 'G20USSD': 679,
 'G20USSCBLE': 7,
 'TOTPOP': 2305,
 'HISP': 383,
 'NH_WHITE': 591,
 'NH_BLACK': 1203,
 'NH_AMIN': 5,
 'NH_ASIAN': 25,
 'NH_NHPI': 1,
 'NH_OTHER': 19,
 'NH_2MORE': 78,
 'VAP': 1623,
 'HVAP': 1534,
 'WVAP': 5,
 'BVAP': 22,
 'AMINVAP': 1,
 'ASIANVAP': 156,
 'NHPIVAP': 89,
 'OTHERVAP': 84,
 '2MOREVAP': 13,
 'G18GOVD': 25.746192893401012,
 'G18GOVR': 0.893401015228426,
 'G18SOSD': 26.233502538071065,
 'G18SOSR': 0.649746192893401,
 'G18TRED': 25.82741116751269,
 'G18TRER': 0.649746192893401,
 'G18ATGD': 25.746192893401012,
 'G18ATGR': 0.568527918781726,
 'G18COMR': 3.32994923857868,
 'G18SPIR': 1.055837563451776,
 'G18AGRR': 1.299492385786802,
 'SEND': '45',
 'geometry': <POLYGON ((523042.065 3587876.474, 523009.381 3587902.893, 522993.181 358791...>}

In [ ]:
## setting up initial markov chain model

## functions for markov chain updaters
def num_cut_edges(partition):
    return len(partition['cut_edges'])

def democratic_wins(partition):
    return sum(
        1 for district in partition.parts 
        if sum(partition.graph.nodes[node]["G18ATGD"] for node in district) > 
           sum(partition.graph.nodes[node]["G18ATGR"] for node in district)
    )

elections = [
    Election("G18ATG", {"Democratic": "G18ATGD", "Republican": "G18ATGR"}),
    Election("G18SOS", {"Democratic": "G18SOSD", "Republican": "G18SOSR"}),
    Election("G18GOV", {"Democratic": "G18GOVD", "Republican": "G18GOVR"}),
    Election("G18TRE", {"Democratic": "G18TRED", "Republican": "G18TRE"})]
election_updaters = {election.name: election for election in elections}

my_updaters = {
        "cut_edges": cut_edges,
        "population": Tally("TOTPOP", alias="population"),
        "democratic_wins": democratic_wins,
        "number_cut_edges": num_cut_edges,
    }
my_updaters.update(election_updaters)

## creating initial partition
initial_partition = Partition(
    sc_graph, 
    assignment="CD",
    updaters=my_updaters
)

## using RECOM on the random walk, need ideal population
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2,
                   method = partial(
                        bipartition_tree,
                        max_attempts=100,
                        allow_pair_reselection=True  # <-- This is the only change
                    )
)


## all used for building markov chain
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)
pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.02)





In [ ]:
## running markov chain random walk (using recom proposal)
num_steps = 500
for i in range(2):
    chain = MarkovChain(
        proposal=proposal,
        constraints=[compactness_bound, pop_constraint],
        accept=accept.always_accept,
        initial_state=initial_partition,
        total_steps=num_steps ## sample step count for now
    )